In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
# data = open("./data/howtogetrich.txt", "r").read()[:50]
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'e': 0, 'l': 1, ' ': 2, 'w': 3, 'h': 4, 'd': 5, 'o': 6, 'r': 7},
 {0: 'e', 1: 'l', 2: ' ', 3: 'w', 4: 'h', 5: 'd', 6: 'o', 7: 'r'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
bh = np.random.randn(1, hidden_size) * 0.01

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([4, 0, 1, 1, 6, 2, 3, 6, 7, 1], [0, 1, 1, 6, 2, 3, 6, 7, 1, 5])

In [9]:
sample("h", 4)

'lddl'

In [10]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    hs[-1] = h
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t - 1] * dh_dz1.T * dL_dh.T
        dL_dbh += dh_dz1 * dL_dh

        # raise RuntimeError

        # dF_dh = dL_dh * (dh_dz1 @ Whh)
        dF_dh = dL_dh @ (dh_dz1 * Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 20.829193884479537
[Loss]: 16.688622016432642
[Loss]: 13.440257812256379
[Loss]: 11.017179581267232
[Loss]: 9.301011020790806
[Loss]: 8.07086772624487
[Loss]: 7.163233028549479
[Loss]: 6.481406239408077
[Loss]: 5.94717856767263
[Loss]: 5.506982477658848
[Loss]: 5.125948627521994
[Loss]: 4.783173106661796
[Loss]: 4.4655834934560845
[Loss]: 4.165039865541143
[Loss]: 3.876417312385815
[Loss]: 3.59691105806394
[Loss]: 3.3253040522451727
[Loss]: 3.0618275961861405
[Loss]: 2.8076581095354762
[Loss]: 2.5647224545054947
[Loss]: 2.3351220351064588
[Loss]: 2.12082425758994
[Loss]: 1.9232216822686887
[Loss]: 1.7430301712725818
[Loss]: 1.5802107020941276
[Loss]: 1.434148032600261
[Loss]: 1.303767602343764
[Loss]: 1.1877725354409292
[Loss]: 1.084745344310406
[Loss]: 0.9932914490092796
[Loss]: 0.9120788001689273
[Loss]: 0.8398868343789266
[Loss]: 0.7756128938621617
[Loss]: 0.718277347516458
[Loss]: 0.6670171742146666
[Loss]: 0.6210773984402109
[Loss]: 0.5798004042446616
[Loss]: 0.54261500015

In [13]:
sample("h", 10)

'ello world'